#Zero-Shot Learning on a Korean NLI Dataset

The idea here is to test the capabilities of our finetuned NLI model on an unseen (on the use case) language. We chose **korean** as its syntax and typography weren't seen in the data we used to finetune the model.

In [ ]:
%pip install -qU pandas torch transformers datasets onnxruntime tqdm torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.7.0 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.


In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_PATH = "ajayat/xlm-roberta-large-xnli"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, ou

We use the KorNLI dataset. It is a Korean Natural Language Inference (NLI) dataset. The dataset is constructed by automatically translating the training sets of the SNLI, XNLI and MNLI datasets. It contains 942,854 training examples translated automatically and 7,500 evaluation (development and test) examples translated manually.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("klue", "nli")
df = dataset["validation"].to_pandas()


README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/224k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
def tokenize_batch(premises, hypotheses, max_length=512): # We need to tokenize, as usual
    return tokenizer(
        premises,
        hypotheses,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )


In [ ]:
def predict(df):
    predictions = []
    batch_size = 32

    for i in tqdm(range(0, len(df), batch_size)):
        batch = df.iloc[i:i+batch_size]
        tokens = tokenize_batch(batch["premise"].tolist(), batch["hypothesis"].tolist())
        input_ids = tokens["input_ids"].to(device)
        attention_mask = tokens["attention_mask"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            predictions.extend(preds)

    return predictions


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

df["prediction"] = predict(df)

if "label" in df.columns:
    print(classification_report(df["label"], df["prediction"], target_names=['entailment', 'neutral', 'contradiction']))
    print("Accuracy:", accuracy_score(df["label"], df["prediction"]))

100%|██████████| 94/94 [00:34<00:00,  2.70it/s]

               precision    recall  f1-score   support

   entailment       0.80      0.95      0.87      1000
      neutral       0.91      0.73      0.81      1000
contradiction       0.86      0.88      0.87      1000

     accuracy                           0.85      3000
    macro avg       0.86      0.85      0.85      3000
 weighted avg       0.86      0.85      0.85      3000

Accuracy: 0.8526666666666667


We also get a pretty good accuracy on an unseen language ! That's probably due to XLM Roberta's pre-training, which was also done on Korean. The finetuning enhances its performance on an NLI task.

Example of misclassified instance :



*   1997년 현대 유니콘스는 선수층 빈약함이 여실히 드러나게 되어 정규 시즌 6위를 기록한다: *In 1997, the Hyundai Unicorns' weakness in player base was clearly revealed, and they finished in 6th place in the regular season*
*   현대 유니콘스는 1997년 정규 시즌 순위에 들지 못한다: *The Hyundai Unicorns failed to qualify for the 1997 regular season*
* Real label: 2 (contradiction)
* Predicted label: 0 (entailment)


Let's try to explain which tokens are responsible for the classification by using the integrated gradients method.

In [ ]:
!pip install -qU captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.


In [ ]:
from captum.attr import LayerIntegratedGradients
import torch

# Make sure model is in eval mode
model.eval()

# We'll use the embeddings layer of XLMRoberta
model_input = model.roberta.embeddings

def model_output(inputs, attention_mask=None):
    outputs = model(inputs, attention_mask=attention_mask)
    return outputs.logits

For the integrated gradients method to work, we need to compute a baseline :

In [ ]:
def construct_input_and_baseline(premise, hypothesis, tokenizer, device):
    max_length = 510  # Leaving space for special tokens

    encoding = tokenizer(
        premise,
        hypothesis,
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
        add_special_tokens=True,
    )

    # Move the input tensors to the specified device
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Baseline: just pad tokens where there are real tokens
    baseline_input_ids = torch.full_like(input_ids, tokenizer.pad_token_id).to(device)

    # Get tokens for visualization (these will remain on the CPU as they are for display purposes)
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())

    return input_ids, attention_mask, baseline_input_ids, tokens

In [ ]:
lig = LayerIntegratedGradients(model_output, model_input)

In [ ]:
def interpret_premise_hypothesis(premise, hypothesis, true_class, target_class, tokenizer,device):
    input_ids, attention_mask, baseline_input_ids, tokens = construct_input_and_baseline(premise, hypothesis, tokenizer,device)

    attributions, delta = lig.attribute(
        inputs=input_ids,
        baselines=baseline_input_ids,
        additional_forward_args=(attention_mask,),
        return_convergence_delta=True,
        internal_batch_size=1,
        target=target_class,
    )

    attributions_sum = attributions.sum(dim=-1).squeeze(0)
    attributions_sum = attributions_sum / torch.norm(attributions_sum)

    from captum.attr import visualization as viz

    pred = model_output(input_ids, attention_mask)

    score_vis = viz.VisualizationDataRecord(
        word_attributions=attributions_sum,
        pred_prob=torch.max(pred),
        pred_class=torch.argmax(pred).cpu().item(),
        true_class=true_class,
        attr_class=f"{premise} [SEP] {hypothesis}",
        attr_score=attributions_sum.sum(),
        raw_input_ids=tokens,
        convergence_score=delta
    )

    viz.visualize_text([score_vis])



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
premise = "1997년 현대 유니콘스는 선수층 빈약함이 여실히 드러나게 되어 정규 시즌 6위를 기록한다."
hypothesis = "현대 유니콘스는 1997년 정규 시즌 순위에 들지 못한다."
true_class = 2
target_class = 0

interpret_premise_hypothesis(premise, hypothesis, true_class, target_class, tokenizer,device)